In [ ]:
from params.general_params import *
from subsystems.simulator import TtsPidSim
from common.utils import *
import pygad

def fitness_func(ga_instance, solution, solution_idx):
    # Symulacja
    system.in_var_val('ke', solution[0])
    system.in_var_val('kdCV', solution[1])
    proc_vars = system.simulate(tend, 'control')
    # Wyznaczenie wskaźników oceny jakości regulacji
    eval = calculate_quality_indicators(dt, proc_vars['t'], sp, proc_vars['e'])
    fitness1 = 1.0 / (eval['st'] + 0.000001)
    fitness2 = 1.0 / (eval['ISE'] + 0.000001)
    progress_bar(solution_idx + 1, sol_per_pop, 1)
    print(f" Step: {solution_idx + 1} Solution: kp={solution[0]} Eval - st: {eval['st']}, ISE: {eval['ISE']}")
    return [fitness1, fitness2]


# Utworzenie obiektu symulacji
system = TtsPidSim(tp, pr, True)

# Parametryzacja
sp = 0.2
tend = 300
system.in_var_val('SPmode', 1)
system.in_var_val('SPman', sp)
#   krok czasowy symulacji
dt = tp*pr['control']
# - regulator rozmyty
system.in_var_val('ke', 0.05)
system.in_var_val('kde', 0.0005)
system.in_var_val('kdCV', 10/dt)

# Wyniki dla wartości domyślnych
proc_vars = system.simulate(tend, 'control')
evaluation = calculate_quality_indicators(dt, proc_vars['t'], sp, proc_vars['e'])
show_evaluation(proc_vars['t'], proc_vars['SP'], proc_vars['PV'], proc_vars['e'], proc_vars['CV'], evaluation)

# Parametry AG
num_generations = 100
num_parents_mating = 10
sol_per_pop = 20
#   dostrajne będą parametry: ke, kdCV
num_genes = 2
init_range_low = 0.01
init_range_high = 10

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       mutation_num_genes=1,
                       fitness_func=fitness_func,
                       parent_selection_type='nsga2',
                       init_range_low=init_range_low,
                       init_range_high=init_range_high)

ga_instance.run()

solution, solution_fitness, solution_idx = ga_instance.best_solution(ga_instance.last_generation_fitness)
print(f"Parameters of the best solution : {solution}")
print(f"Fitness value of the best solution = {solution_fitness}")

# Wyniki dla wartości po optymalizacji
system.in_var_val('ke', solution[0])
system.in_var_val('kdCV', solution[1])
proc_vars = system.simulate(tend, 'control')
evaluation = calculate_quality_indicators(dt, proc_vars['t'], sp, proc_vars['e'])
show_evaluation(proc_vars['t'], proc_vars['SP'], proc_vars['PV'], proc_vars['e'], proc_vars['CV'], evaluation)